# 🎯Aim/Objective

- Take any csv file of size 2 GB or more.
- Perform loading of data using Pandas, Dask, Modin, and Ray. Also, compare the data loading time with each.
- Create YAML file containing the meta data of the dataset such as # of rows, # of columns, column names, file name, file path etc.
- Create utility.py file which performs the validations based on the YAML file.
- Write the final file into a compressed format, eg. .gz format.
- Display summary of the dataset containing # of rows, columns andfile size.

# ⤵️Import Libraries

In [1]:
import os
import re
import yaml
import time
import logging
import pandas as pd
import dask.dataframe as dd
import ray as ry
import modin.pandas as mpd

In [2]:
file_path = os.path.join(os.getcwd(), 
                         'Dataset/archive/game_dataset.csv')

# 📖Different approach of reading file

## Using Pandas

In [3]:
start_time = time.process_time()
pandas_df = pd.read_csv(file_path, 
                        sep=',')
end_time = time.process_time()
print(r'Pandas file read efficiency: {}sec'.format(end_time - start_time))

Pandas file read efficiency: 94.15625sec


## Using Dask 

In [4]:
start_time = time.process_time()
dask_df = dd.read_csv(file_path, 
                      sep=',')
end_time = time.process_time()
print(r'Dask file read efficiency: {}sec'.format(end_time - start_time))

Dask file read efficiency: 0.03125sec


## Using Modin & Ray

In [5]:
ry.shutdown()
ry.init()

start_time = time.process_time()
modin_df = mpd.read_csv(file_path, 
                        sep=',')
end_time = time.process_time()
print(r'Modin & Ray file read efficiency: {}sec'.format(end_time - start_time))

2023-11-07 12:34:22,561	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Modin & Ray file read efficiency: 8.84375sec


As you can see, using **Dask** is clearly the best option for reading the csv file as the loading time - **0.03 sec** is less as compared to others.

# ✍️Write YAML file

In [10]:
%%writefile schema.yaml

file_type: csv
file_name: game_dataset
file_path: 'Dataset/archive/game_dataset.csv'
inbound_delimiter: ","
outbound_delimiter: "|"
columns: 
    - app_id
    - app_name
    - review_id
    - language
    - review
    - timestamp_created
    - timestamp_updated
    - recommended
    - votes_helpful

Overwriting schema.yaml


# 📂Generate Utility file

In [11]:
%%writefile utility_val.py

import os
import re
import yaml
import time
import logging
import pandas as pd

def read_yaml_file(file):
    with open(file, 'r') as f:
        try:
            return yaml.safe_load(f)
        except yaml.YAMLError as e:
            logging.error(e)
            
def validate(df, config_data):
    
    if 'columns' not in config_data:
        print('Error: Columns dictionary missing in the config file. Can\'t validate')
    else:
        config_cols = sorted(config_data['columns'])
        df_cols = sorted(df.columns)
        
        # Remove leading or trailing white spaces
        df.columns = list(map(lambda x:x.strip(), 
                              list(df.columns)))
        
        # Convert to lowercase
        df.columns = list(map(lambda x:x.lower(), 
                              list(df.columns)))
        
        # Remove any special characters
        df.columns = list(map(lambda x:re.sub('[^a-z_]+', '', x) if x != 'Unnamed: 0' else x, 
                              list(df.columns)))
        
        if len(config_cols) != len(df_cols):
            print('Error: Invalid number of columns in either config_file/dataset.')
            return 0
        elif list(config_cols) != list(df_cols):
            print('Error: Column names not matching as per config file.')
            return 0
        else:
            print('All tests passed.')
            return 1

Overwriting utility_val.py


In [3]:
import utility_val as util

config_data = util.read_yaml_file(os.path.join(os.getcwd(), 
                                               'schema.yaml'))
config_data

{'file_type': 'csv',
 'file_name': 'game_dataset',
 'file_path': 'Dataset/archive/game_dataset.csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'columns': ['app_id',
  'app_name',
  'review_id',
  'language',
  'review',
  'timestamp_created',
  'timestamp_updated',
  'recommended',
  'votes_helpful']}

In [4]:
config_data['file_path'] = config_data['file_path'].replace('/', '\\')
file_path = os.path.join(os.getcwd(), 
                         config_data['file_path'])
delimeter = str(config_data['inbound_delimiter'])
pandas_df = pd.read_csv(file_path, 
                        sep=delimeter)
pandas_df.head()

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0


# ✅Validation

In [5]:
util.validate(pandas_df, 
              config_data)

Error: Invalid number of columns in either config_file/dataset.


0

In [6]:
pandas_df.columns

Index(['unnamed', 'app_id', 'app_name', 'review_id', 'language', 'review',
       'timestamp_created', 'timestamp_updated', 'recommended',
       'votes_helpful'],
      dtype='object')

In [7]:
config_data['columns']

['app_id',
 'app_name',
 'review_id',
 'language',
 'review',
 'timestamp_created',
 'timestamp_updated',
 'recommended',
 'votes_helpful']

In [8]:
# Remove the 1st column
pandas_df.drop('unnamed', 
               axis=1, 
               inplace=True)
util.validate(pandas_df, 
              config_data)

All tests passed.


1

# 💾Save the file in 'gz' format

In [9]:
outbound_del = config_data['outbound_delimiter']
outbound_del

'|'

In [10]:
pandas_df.to_csv('outfile.csv.gz', 
                 sep=outbound_del, 
                 compression='gzip', 
                 index=False)

# Summary of Dataset

In [18]:
print('No. of Columns: {}'.format(len(pandas_df.columns)))
print('No. of Rows: {}'.format(len(pandas_df)))
print('Outfile compressed file size: {} GB'.format((os.stat('outfile.csv.gz')).st_size / (1024 * 1024 * 1024)))

No. of Columns: 9
No. of Rows: 21747371
Outfile compressed file size: 2.247827772051096 GB


Due to large compressed file size, it can't be uploaded to git